In [1]:
from pyparticle.population import build_population
from pathlib import Path
import os
import numpy as np

model_output_root = Path.cwd() / "example_data" / "model_output"

timestep = 1 # initial conditions
partmc_dir = model_output_root / "partmc"
partmc_population_cfg = {
        'type':'partmc',
        'partmc_dir': partmc_dir,
        'timestep':timestep,
        'repeat':1, # number of PartMC repeat, if run multiple times
        }
partmc_population = build_population(partmc_population_cfg)

D_min = 1e-9
D_max = 1e-4
N_bins = 300 # number of bins used to analyze MAM4 output
GSDs = [1.8, 1.6, 1.6, 1.8] # geometric standard deviations for each mod
mam4_output_filename = model_output_root / "mam4" / "mam_output.nc"

mam4_population_cfg = {
            'type':'mam4',
            'mam4_dir': model_output_root / "mam4",
            'timestep':timestep,
            'GSD':GSDs, 
            'GMD_init':[1.1e-7, 2.6e-8, 2e-6, 5e-8], # initial GMDs for each mode (will be overwritten if reading from namelist)
            'D_min':1e-9, # minimum diameter for analysis
            'D_max':1e-4, # maximum diameter for analysis 
            'N_bins':N_bins, # number of bins to use for analysis
            'T':293.15, # temperature for analysis
            'p':101325., # pressure for analysis
                }

mam4_population = build_population(mam4_population_cfg)

mfpom2 is not a MAM input parameter; returning 0.0
mfso44 is not a MAM input parameter; returning 0.0


In [ ]:
from pyparticle.viz.style import StyleManager, Theme
from pyparticle.viz.builder import build_plotter
import matplotlib.pyplot as plt

mgr = StyleManager(Theme(), deterministic=False)

series = [
    {"key": "partmc", "population": partmc_population, "label": "PartMC-MOSAIC"},
    {"key": "mam4", "population": mam4_population, "label": "MAM4"},
]
line_styles = mgr.plan("line", [s["key"] for s in series])

fig, ax = plt.subplots()
for s in series:
    cfg = {
        "varname": "frac_ccn",
        "s_grid": np.logspace(-2,1,100),  # supersaturation grid from 0.01% to 10%
        "style": line_styles[s["key"]],
    }
    plotter = build_plotter("state_line", cfg)
    plotter.plot(s["population"], ax, label=s["label"])
ax.legend(); fig.tight_layout()

/Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/pyparticle/src/pyparticle/aerosol_particle.py:260: UserWarning: Surface tension not implemented; returning default 0.072 N/m
  warnings.warn("Surface tension not implemented; returning default 0.072 N/m", UserWarning)
